In [ ]:
aoi = [{'path': 'aoi.geojson'}] # type: stage-in
granules = [{'path': '/path/to/granule.h5'}] # type: stage-in
columns = 'agbd, agbd_se, l2_quality_flag, l4_quality_flag, sensitivity, sensitivity_a2, lon_lowestmode, lan_lowestmode' # type: string
query = '' # type: string
lat = 'lat_lowestmode' # type: string
lon = 'lon_lowestmode' # type: string
beams = 'all' # type: string

Above, the `stage-in` parameter `aoi` is expected to be a list of dictionaries, each of which describes one of the files supplied as the value of this parameter.

If the input value was only a single string, then the value of `aoi` will be an array of size 1 with said dictionary. Take a look at the `inputs.json` supplied to `papermill` for an example of what this dictionary would look like:

```
=================
***inputs.json***
=================
{
    "aoi": [
        {
            "basename": "0_geoBoundaries-GAB-ADM0.geojson",
            "checksum": "sha1$001e7d353bd8143e1315d30ab6aed44139c8f344",
            "class": "File",
            "dirname": "/var/lib/cwl/stg36ad3228-566a-4426-90f7-ed42f53bb4e0",
            "http://commonwl.org/cwltool#generation": 0,
            "location": "file:///data/home/hysdsops/zhan/artifact-deposit-repo/jplzhan/gedi-subset/main/output-nbjb75r8/inputs/0/0_geoBoundaries-GAB-ADM0.geojson",
            "nameext": ".geojson",
            "nameroot": "geoBoundaries-GAB-ADM0",
            "path": "/var/lib/cwl/stg36ad3228-566a-4426-90f7-ed42f53bb4e0/0_geoBoundaries-GAB-ADM0.geojson",
            "size": 51350
        }
    ],
    "granules": [
        {
            "basename": "0_GEDI04_A_2019146134206_O02558_01_T05641_02_002_02_V002.h5",
            "checksum": "sha1$feb08ef019d91ff07e1580c9a849e6e4f28e9fe3",
            "class": "File",
            "dirname": "/var/lib/cwl/stg763e8dd7-fd05-487a-ba3a-cadc53c9c332",
            "http://commonwl.org/cwltool#generation": 0,
            "location": "file:///data/home/hysdsops/zhan/artifact-deposit-repo/jplzhan/gedi-subset/hec/output-p_ok_184/inputs/0/0_GEDI04_A_2019146134206_O02558_01_T05641_02_002_02_V002.h5",
            "nameext": ".h5",
            "nameroot": "0_GEDI04_A_2019146134206_O02558_01_T05641_02_002_02_V002",
            "path": "/var/lib/cwl/stg763e8dd7-fd05-487a-ba3a-cadc53c9c332/0_GEDI04_A_2019146134206_O02558_01_T05641_02_002_02_V002.h5",
            "size": 109948857
        },
        {
            "basename": "1_GEDI04_A_2019160192444_O02779_04_T02282_02_002_02_V002.h5",
            "checksum": "sha1$b73e33aa701ad62d9e41e19c135c8ee264dea12a",
            "class": "File",
            "dirname": "/var/lib/cwl/stg5cb2a269-b7fa-4614-a06c-da98019acbce",
            "http://commonwl.org/cwltool#generation": 0,
            "location": "file:///data/home/hysdsops/zhan/artifact-deposit-repo/jplzhan/gedi-subset/hec/output-p_ok_184/inputs/1/1_GEDI04_A_2019160192444_O02779_04_T02282_02_002_02_V002.h5",
            "nameext": ".h5",
            "nameroot": "1_GEDI04_A_2019160192444_O02779_04_T02282_02_002_02_V002",
            "path": "/var/lib/cwl/stg5cb2a269-b7fa-4614-a06c-da98019acbce/1_GEDI04_A_2019160192444_O02779_04_T02282_02_002_02_V002.h5",
            "size": 216236480
        }
    ],
    "columns": "agbd, agbd_se, l2_quality_flag, l4_quality_flag, sensitivity, sensitivity_a2, lon_lowestmode, lat_lowestmode",
    "maappgt": "PGT-945-qCOwrb6hBV3pNSVyFhwGA9UTeAIRzGzfZf3tbY6uSb-hpRX4G8Y1pyujdKAMjY0WA7U-904a6371b74a",
    "query": "l2_quality_flag == 1 and l4_quality_flag == 1 and sensitivity > 0.95 and sensitivity_a2 > 0.95"
}
```

In general, we only care about the `path` value for our dictionary element within the list defined by `aoi`. This is the filename accessible by our docker container when the job actually runes.

In this version of the notebook, we include an additional `stage-in` parameter called `granules`. This is the list of `.h5` files to be staged in before the execution of the notebook.

In [ ]:
import os

# Collapse the list into a single filepath
aoi_path = aoi[0]['path']

# Write the list of files downloaded by stage-in to a manifest file
manifest_file = os.path.join(os.getcwd(), 'manifest.txt')
with open(manifest_file, 'w') as f:
    for file_desc in granules:
        f.write('{}\n'.format(file_desc['path']))

In [ ]:
%%bash -s "$aoi_path" "$columns" "$query" "$manifest_file" "$lat" "$lon" "$beams"
# write a bash script here
conda run --no-capture-output -n gedi_subset /home/jovyan/gedi-subsetter/src/gedi_subset/subset.py \
    --verbose \
    --aoi "${1}" \
    --columns "${2}" \
    --query "${3}" \
    --granules "${4}" \
    --lat "${5}" \
    --lon "${6}" \
    --beams "${7}"

Couldn't find program: 'bash'
